In [1]:
import requests
from bs4 import BeautifulSoup
import os
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from keras import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
import pandas_datareader as pdr
from pandas import DataFrame

In [2]:
def fileCode(filename):
    KRX = pd.read_csv(os.getcwd() + '\\' + filename + '.csv')
    KRX = KRX[['2017/12', '2018/12', '2019/12', 'code']]
                      
    return KRX

In [3]:
def financialStatements(code):
    s = requests.Session()
    url_test = 'https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode='
    url_test += code
    url_test += '&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701'
    r = s.get(url_test)
    r.encoding = 'utf-8'
    data = pd.read_html(r.text)
    
    IS_temp = data[0]

    if IS_temp.columns[0] == 'IFRS(연결)':
        IS_temp.index = IS_temp['IFRS(연결)'].values
        if IS_temp.columns[1] == '2017/12':
            IS_temp.drop(['IFRS(연결)', '2017/12', '2018/12', '2019/12', '전년동기', '전년동기(%)'], inplace=True, axis=1)
        elif IS_temp.columns[1] == '2018/03':
            IS_temp.drop(['IFRS(연결)', '2018/03', '2019/03', '2020/03', '전년동기', '전년동기(%)'], inplace=True, axis=1)
        elif IS_temp.columns[1] == '2018/12':
            if IS_temp.columns[2] == '2019/06':
                IS_temp.drop(['IFRS(연결)', '2018/12', '2019/06', '2020/06', '전년동기', '전년동기(%)'], inplace=True, axis=1)
            elif IS_temp.columns[3] == '2020/06':
                IS_temp.drop(['IFRS(연결)', '2018/12', '2019/12', '2020/06', '전년동기', '전년동기(%)'], inplace=True, axis=1)
    else:
        IS_temp.index = IS_temp['IFRS(개별)'].values
        IS_temp.drop(['IFRS(개별)', '전년동기', '전년동기(%)'], inplace=True, axis=1)
    
    IS_temp.drop(['법인세비용', '계속영업이익', '중단영업이익'], inplace=True, axis=0)
    
    for i, name in enumerate(IS_temp.index):
        if '참여한' in name:
            name = name.strip().replace('계산에 참여한 계정 펼치기','')
            name = name.replace(' ','')
            IS_temp.rename(index = {str(IS_temp.index[i]): str(name)}, inplace = True)
    
    return IS_temp

In [4]:
def financialStatements1(code):
    s = requests.Session()
    url_test = 'https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode='
    url_test += code
    url_test += '&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701'
    r = s.get(url_test)
    r.encoding = 'utf-8'
    data = pd.read_html(r.text)
    
    IS_temp = data[0]

    if IS_temp.columns[0] == 'IFRS(연결)':
        IS_temp.index = IS_temp['IFRS(연결)'].values
        if IS_temp.columns[1] == '2017/12':
            IS_temp.drop(['IFRS(연결)', '2020/12', '전년동기', '전년동기(%)'], inplace=True, axis=1)
        elif IS_temp.columns[1] == '2018/03':
            IS_temp.rename(columns= {'2018/03':'2017/12', '2019/03':'2018/12', '2020/03':'2019/12'}, inplace=True)
            IS_temp.drop(['IFRS(연결)', '2020/12', '전년동기', '전년동기(%)'], inplace=True, axis=1)
        elif IS_temp.columns[1] == '2018/12':
            if IS_temp.columns[2] == '2019/06':
                IS_temp.rename(columns= {'2018/12':'2017/12', '2019/06':'2018/12', '2020/06':'2019/12'}, inplace=True)
                IS_temp.drop(['IFRS(연결)', '2020/12', '전년동기', '전년동기(%)'], inplace=True, axis=1)
            else:
                IS_temp.rename(columns= {'2018/12':'2017/12', '2019/12':'2018/12', '2020/06':'2019/12'}, inplace=True)
                IS_temp.drop(['IFRS(연결)', '2020/12', '전년동기', '전년동기(%)'], inplace=True, axis=1)
    else:
        if IS_temp.columns[1] == '2017/12':
            IS_temp.index = IS_temp['IFRS(개별)'].values
            IS_temp.drop(['IFRS(개별)', '2020/12', '전년동기', '전년동기(%)'], inplace=True, axis=1)
        elif IS_temp.columns[1] == '2018/06':
            IS_temp.index = IS_temp['IFRS(개별)'].values
            IS_temp.rename(columns= {'2018/06':'2017/12', '2019/06':'2018/12', '2020/06':'2019/12'}, inplace=True)
            IS_temp.drop(['IFRS(개별)', '2020/12', '전년동기', '전년동기(%)'], inplace=True, axis=1)
    
    IS_temp.drop(['법인세비용', '계속영업이익', '중단영업이익'], inplace=True, axis=0)
    
    for i, name in enumerate(IS_temp.index):
        if '참여한' in name:
            name = name.strip().replace('계산에 참여한 계정 펼치기','')
            name = name.replace(' ','')
            IS_temp.rename(index = {str(IS_temp.index[i]): str(name)}, inplace = True)
    
    return IS_temp

In [5]:
KRX = fileCode('주가변동률(기계,장비)')
df = pd.DataFrame()
df = df.transpose()

for i in range(len(KRX)):
    fs = financialStatements1(KRX['code'].values[i])
    fs = fs.transpose()
    ns = KRX.loc[[i], :].transpose()
    ns.drop(['code'], inplace=True)
    ns.columns = ["주가변동률"]
    fs = pd.concat([fs, ns], axis=1)
    df = pd.concat([df, fs])
    
df = df.fillna(0)
df

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


,금융수익,금융원가,기타비용,기타수익,당기순이익,매출액,매출원가,매출총이익,비지배주주순이익,세전계속사업이익,영업이익,영업이익(발표기준),"종속기업,공동지배기업및관계기업관련손익",주가변동률,지배주주순이익,판매비와관리비
2017/12,989.0,1172.0,35.0,40.0,-103.0,7689.0,7139.0,550.0,0.0,-49.0,135.0,135.0,-6.0,105.527443,-103.0,415.0
2018/12,576.0,644.0,4.0,133.0,-187.0,5113.0,5094.0,19.0,0.0,-296.0,-353.0,-353.0,-3.0,-15.340909,-187.0,372.0
2019/12,687.0,863.0,66.0,3.0,-414.0,6743.0,6636.0,107.0,0.0,-457.0,-218.0,-218.0,-0.0,87.919463,-414.0,325.0
2017/12,3.0,5.0,90.0,129.0,69.0,1685.0,1465.0,220.0,2.0,93.0,35.0,35.0,22.0,-11.064718,67.0,186.0
2018/12,3.0,5.0,59.0,89.0,22.0,1820.0,1590.0,230.0,-20.0,27.0,37.0,37.0,-38.0,60.798122,42.0,193.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018/12,37.0,37.0,28.0,0.0,-190.0,307.0,280.0,27.0,0.0,-181.0,-81.0,-81.0,-72.0,7.389163,-190.0,108.0
2019/12,7.0,17.0,27.0,1.0,-95.0,176.0,151.0,26.0,-2.0,-95.0,-52.0,-52.0,-8.0,59.174312,-93.0,77.0
2017/12,0.0,2.0,4.0,6.0,70.0,916.0,768.0,148.0,0.0,88.0,88.0,88.0,0.0,81.303116,70.0,60.0
2018/12,0.0,3.0,10.0,10.0,83.0,1180.0,1001.0,179.0,0.0,109.0,111.0,111.0,0.0,-30.859375,83.0,68.0


In [6]:
KRX1 = fileCode('주가변동률(기계,장비)')
df1 = pd.DataFrame()
df1 = df1.transpose()

for n in range(len(KRX1)):
    fs1 = financialStatements(KRX1['code'].values[n])
    fs1 = fs1.transpose()
    df1 = pd.concat([df1, fs1])

df1 = df1.fillna(0)
df1.drop(['2017/12', '2018/12', '2019/12', '2018/06', '2019/06', '2020/06'], inplace = True ,axis=0)
df1

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,금융수익,금융원가,기타비용,기타수익,당기순이익,매출액,매출원가,매출총이익,비지배주주순이익,세전계속사업이익,영업이익,영업이익(발표기준),"종속기업,공동지배기업및관계기업관련손익",지배주주순이익,판매비와관리비
2020/12,805.0,978.0,9.0,70.0,60.0,8300.0,7767.0,533.0,0.0,87.0,200.0,200.0,-1.0,60.0,333.0
2020/12,5.0,18.0,306.0,132.0,-639.0,3279.0,3428.0,-149.0,3.0,-631.0,-447.0,-447.0,3.0,-642.0,298.0
2020/12,324.0,217.0,433.0,73.0,579.0,16003.0,14107.0,1897.0,0.0,389.0,637.0,637.0,4.0,579.0,1259.0
2020/12,319.0,127.0,29.0,189.0,341.0,3659.0,3282.0,377.0,0.0,429.0,78.0,78.0,-1.0,341.0,299.0
2020/12,57.0,154.0,0.0,2.0,98.0,2374.0,1973.0,401.0,0.0,136.0,231.0,231.0,0.0,98.0,170.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020/12,318.0,78.0,68.0,15.0,80.0,157.0,165.0,-8.0,-29.0,101.0,-87.0,-87.0,0.0,108.0,79.0
2020/12,6.0,11.0,13.0,5.0,113.0,1975.0,1613.0,361.0,0.0,154.0,166.0,166.0,1.0,111.0,196.0
2020/12,3.0,4.0,2.0,0.0,12.0,589.0,497.0,92.0,0.0,15.0,18.0,18.0,0.0,0.0,75.0
2020/12,480.0,41.0,2.0,111.0,390.0,207.0,178.0,29.0,-2.0,487.0,-89.0,-89.0,27.0,392.0,118.0


In [8]:
feature_cols = ['매출액', '영업이익', '당기순이익', '매출총이익']

X = Normalizer().fit_transform(df[feature_cols])
y = df["주가변동률"].apply(lambda x : 1 if x > 24 else 0)

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    shuffle=True,
)

model = Sequential()
model.add(Dense(4, activation="relu", input_shape=(X_train.shape[1],)))
model.add(Dense(1, activation="sigmoid"))
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])
model.fit(X_train, y_train, epochs=100)

y_pred = model.predict(X_test)

Epoch 1/100
11/11 [==============================] - 1s 2ms/step - loss: 0.5921 - acc: 0.7586
Epoch 2/100
11/11 [==============================] - 0s 3ms/step - loss: 0.5974 - acc: 0.7394
Epoch 3/100
11/11 [==============================] - 0s 1ms/step - loss: 0.5957 - acc: 0.7382
Epoch 4/100
11/11 [==============================] - 0s 1ms/step - loss: 0.5829 - acc: 0.7482
Epoch 5/100
11/11 [==============================] - 0s 1ms/step - loss: 0.5650 - acc: 0.7694
Epoch 6/100
11/11 [==============================] - 0s 1ms/step - loss: 0.5842 - acc: 0.7379
Epoch 7/100
11/11 [==============================] - 0s 1ms/step - loss: 0.5819 - acc: 0.7413
Epoch 8/100
11/11 [==============================] - 0s 1ms/step - loss: 0.5686 - acc: 0.7595
Epoch 9/100
11/11 [==============================] - 0s 1ms/step - loss: 0.5671 - acc: 0.7508
Epoch 10/100
11/11 [==============================] - 0s 2ms/step - loss: 0.5865 - acc: 0.7318
Epoch 11/100
11/11 [==============================] - 0s 1m

In [69]:
y_pred

array([[0.2906624 ],
       [0.29648575],
       [0.2958742 ],
       [0.29776603],
       [0.291399  ],
       [0.290174  ],
       [0.29284608],
       [0.30260447],
       [0.3007953 ],
       [0.2921431 ],
       [0.29914278],
       [0.29785407],
       [0.29241621],
       [0.29506797],
       [0.29218295],
       [0.33314395],
       [0.3346163 ],
       [0.37701073],
       [0.3495524 ],
       [0.29618305],
       [0.29697716],
       [0.29335088],
       [0.29845855],
       [0.29535043],
       [0.29941964],
       [0.3037941 ],
       [0.2929961 ],
       [0.30775976],
       [0.30114728],
       [0.3260015 ],
       [0.4046118 ],
       [0.29668486],
       [0.38417697],
       [0.29377794],
       [0.29886395],
       [0.3726408 ],
       [0.29069993],
       [0.28892952],
       [0.28917485],
       [0.29439664],
       [0.29656285],
       [0.32638478],
       [0.29658616],
       [0.28998387],
       [0.29573795],
       [0.30123252],
       [0.2990182 ],
       [0.462

In [77]:
X1 = Normalizer().fit_transform(df1[feature_cols])

y_pred1 = model.predict(X1)
print(y_pred1)

[[0.37098163]
 [0.35682052]
 [0.3676091 ]
 [0.3938908 ]
 [0.34802163]
 [0.37505323]
 [0.36488205]
 [0.36485928]
 [0.35435748]
 [0.36878175]
 [0.34990454]
 [0.3451206 ]
 [0.34845638]
 [0.3479191 ]
 [0.341106  ]
 [0.34068954]
 [0.35783213]
 [0.34353572]
 [0.30153129]
 [0.2511593 ]
 [0.34647387]
 [0.34518385]
 [0.34789366]
 [0.25885716]
 [0.36548996]
 [0.35311294]
 [0.3426234 ]
 [0.28485733]
 [0.33633533]
 [0.34275815]
 [0.34572294]
 [0.38463366]
 [0.36036545]
 [0.34700012]
 [0.32218975]
 [0.35640222]
 [0.3584032 ]
 [0.34758264]
 [0.34668314]
 [0.36927497]
 [0.36471403]
 [0.35970947]
 [0.35376936]
 [0.36153448]
 [0.36064407]
 [0.28756547]
 [0.37083328]
 [0.29794794]
 [0.21441007]
 [0.37925702]
 [0.36885554]
 [0.347099  ]
 [0.3368851 ]
 [0.34917372]
 [0.28915608]
 [0.31033874]
 [0.28596324]
 [0.3521718 ]
 [0.29416597]
 [0.34082007]
 [0.34939498]
 [0.27776742]
 [0.3670354 ]
 [0.3074149 ]
 [0.35212815]
 [0.33122098]
 [0.34227186]
 [0.3565924 ]
 [0.33845484]
 [0.3757981 ]
 [0.3705648 ]
 [0.36

In [89]:
final = pd.DataFrame(y_pred1)
final.to_csv('D:\\영우\\강의자료\\4-1\\AI주식\\final.csv')